<a href="https://colab.research.google.com/github/gloguercio/cloth_classifier/blob/main/DeepLearning_Track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import opendatasets as od
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import BatchNormalization ,GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau ,ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
# Downloading the dataset from Kaggle (assuming `od` is properly imported)
od.download('https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small',data_dir="dataset")

In [ ]:
# Load the CSV file containing information about fashion products into a Pandas DataFrame
clothes_df = pd.read_csv('dataset/fashion-product-images-small/styles.csv', on_bad_lines='skip')

In [ ]:
# Display the first few rows of the DataFrame
clothes_df.head()

In [ ]:
# Remove rows with missing values
clothes_df = clothes_df.dropna()

In [ ]:
# Get the number of unique values in each column
clothes_df.nunique()

In [ ]:
# Display the column names
clothes_df.columns

In [ ]:
# Combine different categorical columns into one for creating labels
clothes_df['combined_category'] = clothes_df['articleType'] + '_' + clothes_df['baseColour'] + '_' + clothes_df['masterCategory'] + '_' + clothes_df['subCategory']

In [ ]:
# Get the length of the DataFrame after data cleaning
len(clothes_df)

In [ ]:
# Create an empty list to store image data
data = []

In [ ]:
# Loop through each product ID and process its associated image
for name in clothes_df.id:
    try:
        # Read the image
        image = cv2.imread('dataset/fashion-product-images-small/images/'+str(name)+'.jpg')
        # Resize the image to a fixed size
        image = cv2.resize(image, (IX,IY) )
        # Convert the image to a NumPy array
        image = img_to_array(image)
        # Append the image data to the list
        data.append(image)
    except:
        # Skip images for which there are errors in loading
        invalid_ids.append(name)

In [ ]:
# Create an empty list to store labels
labels = []

In [ ]:
# Iterate through each row in the DataFrame to extract labels
for index, row in clothes_df.iterrows():
    if row['id'] in invalid_ids:
        continue
    tags = []
    for col in used_column:
        tags.append(row[col])
    labels.append(tags)

In [ ]:
# Convert the lists to NumPy arrays
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
# Print the extracted labels
print(labels)

In [ ]:
# Binarize the labels using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

In [ ]:
# Print the classes learned by the MultiLabelBinarizer
print(mlb.classes_)
print(labels[0])

In [ ]:
# Define the input shape for the model
inputShape = (60, 80, 3)

In [ ]:
# Define the architecture of the neural network model
model = Sequential([
    Conv2D(32, (3, 3), padding="same", input_shape=inputShape),
    Activation("relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='sigmoid'),
    Dense(labels.shape[1], activation='sigmoid')  # Output layer with the number of unique combined categories
])

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Split the dataset into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.1, random_state=42)

In [ ]:
# Set batch size
batch = 32

In [ ]:
# Train the model
history = model.fit(x=trainX, y=trainY, epochs=50, batch_size=32, verbose=1)

In [ ]:
# Make predictions on the testing set
preds = model.predict(testX)

In [ ]:
# Evaluate the model on the testing set
accuracy = model.evaluate(testX, testY, verbose=0)[1]
print("Accuracy: ", accuracy)

In [ ]:
# Print the binarized labels
print(labels)